In [1]:
import numpy as np
import pandas as pd

In [20]:
sonnets = []
with open ("spenser.txt", "r") as myfile:
    data=myfile.readlines()
    data = list(filter(lambda x: x != '\n', data))
    i = 1
    while i < len(data):
        sonnets.append(data[i:i+14])
        i += 15
    for i, sonnet in enumerate(sonnets):
        snt = ''
        for line in sonnet:
            snt += line.strip() + ' '
        snt = snt.replace(',', ' ,').replace('.', ' .').replace(':', ' :').replace(';', ' ;').replace('!', ' !').replace('?', ' ?')
        sonnets[i] = snt[:-1].lower().split()
    print(sonnets[1])

['unquiet', 'thought', ',', 'whom', 'at', 'the', 'first', 'i', 'bred', ',', 'of', "th'", 'inward', 'bale', 'of', 'my', 'love-pined', 'heart', ':', 'and', 'sithens', 'have', 'with', 'sighs', 'and', 'sorrows', 'fed', ',', 'till', 'greater', 'then', 'my', 'womb', 'thou', 'woxen', 'art', '.', 'break', 'forth', 'at', 'length', 'out', 'of', 'the', 'inner', 'part', ',', 'in', 'which', 'thou', 'lurkest', 'like', 'to', "viper's", 'brood', ':', 'and', 'seek', 'some', 'succour', 'both', 'to', 'ease', 'my', 'smart', 'and', 'also', 'to', 'sustain', 'thy', 'self', 'with', 'food', '.', 'but', 'if', 'in', 'presence', 'of', 'that', 'fairest', 'proud', 'thou', 'chance', 'to', 'come', ',', 'fall', 'lowly', 'at', 'her', 'feet', ':', 'and', 'with', 'meek', 'humbless', 'and', 'afflicted', 'mood', ',', 'pardon', 'for', 'thee', ',', 'and', 'grace', 'for', 'me', 'entreat', '.', 'which', 'if', 'she', 'grant', ',', 'then', 'live', ',', 'and', 'my', 'love', 'cherish', ',', 'if', 'not', ',', 'die', 'soon', ',', 'a

In [10]:
dict_words = pd.read_csv('Syllable_dictionary.txt', header = None)

In [11]:
syllables = {}
for i in range(len(dict_words)):
    row = dict_words[0][i]
    word = ''
    for j in range(len(row)):
        if(row[j]!=' '):
            word = word+row[j]
        else:
            syllable_list = []
            current_string = ''
            for j1 in range(j+1, len(row)):
                if(row[j1] == ' '):
                    syllable_list.append(current_string)
                    current_string = ''
                else:
                    current_string = current_string + row[j1]
                    if(j1 == len(row) - 1):
                        syllable_list.append(current_string)
                        
            syllable_list_2 = []     
            for s in syllable_list:
                if(len(s) == 1):
                    syllable_list_2.append(int(s))
                else:
                    syllable_list_2.append(int(s[1]))
            syllables[word] = syllable_list_2
            break
            

[2]

In [13]:
raw_data = pd.read_csv('shakespeare.txt',header=None,sep='\n')
raw_data.head()

,0
0,1
1,"From fairest creatures we desire increase,"
2,"That thereby beauty's rose might never die,"
3,"But as the riper should by time decease,"
4,His tender heir might bear his memory:


In [29]:
sonnets = []
sonnet = ''
for i in range(1,len(raw_data)):
    if(len(raw_data[0][i].strip()) < 4):
        sonnets.append(sonnet)
        sonnet = ''
    elif(i == len(raw_data) - 1):
        sonnet = sonnet + raw_data[0][i] + '\n'
        sonnets.append(sonnet)
    else:
        sonnet = sonnet + raw_data[0][i] + '\n'
print(len(sonnets))

154


In [30]:
print(sonnets[153])

The little Love-god lying once asleep,
Laid by his side his heart-inflaming brand,
Whilst many nymphs that vowed chaste life to keep,
Came tripping by, but in her maiden hand,
The fairest votary took up that fire,
Which many legions of true hearts had warmed,
And so the general of hot desire,
Was sleeping by a virgin hand disarmed.
This brand she quenched in a cool well by,
Which from Love's fire took heat perpetual,
Growing a bath and healthful remedy,
For men discased, but I my mistress' thrall,
  Came there for cure and this by that I prove,
  Love's fire heats water, water cools not love.



In [31]:
for i, sonnet in enumerate(sonnets):
    sonnets[i] = sonnet.lower()

In [32]:
print(sonnets[153])

the little love-god lying once asleep,
laid by his side his heart-inflaming brand,
whilst many nymphs that vowed chaste life to keep,
came tripping by, but in her maiden hand,
the fairest votary took up that fire,
which many legions of true hearts had warmed,
and so the general of hot desire,
was sleeping by a virgin hand disarmed.
this brand she quenched in a cool well by,
which from love's fire took heat perpetual,
growing a bath and healthful remedy,
for men discased, but i my mistress' thrall,
  came there for cure and this by that i prove,
  love's fire heats water, water cools not love.



In [39]:
syllables[','] = [0]
syllables['.'] = [0]
syllables['!'] = [0]
syllables['?'] = [0]
syllables[':'] = [0]
syllables[';'] = [0]

In [40]:
states = list(syllables.keys())

In [41]:
print(len(states))

3212


In [60]:
def sonnet_to_indices(s):
    s = s.replace('\n', ' ').replace(',', ' ,').replace('.', ' .').replace(':', ' :').replace(';', ' ;').replace('!', ' !').replace('?', ' ?')
    results = list(filter(lambda e: e != '', s.split(' ')))
    for i, j in enumerate(results):
        results[i] = states.index(j)
    return results

In [62]:
in

[1109,
 936,
 574,
 3025,
 692,
 1403,
 3205,
 2719,
 2733,
 222,
 2262,
 1696,
 1787,
 716,
 3205,
 391,
 151,
 2721,
 2247,
 2405,
 393,
 2784,
 640,
 3205,
 1323,
 2707,
 1294,
 1696,
 210,
 1323,
 1682,
 3210,
 391,
 2753,
 535,
 2793,
 2741,
 1895,
 364,
 926,
 3205,
 983,
 2775,
 1559,
 1024,
 3116,
 2347,
 1116,
 3205,
 1642,
 5,
 948,
 3057,
 14,
 1554,
 3205,
 2775,
 2339,
 2775,
 1041,
 3205,
 2793,
 2775,
 2661,
 2339,
 2807,
 588,
 3210,
 2753,
 2719,
 148,
 1816,
 2721,
 3150,
 1104,
 1871,
 3205,
 113,
 1861,
 1302,
 2793,
 2721,
 1133,
 2533,
 3205,
 3119,
 2741,
 1895,
 376,
 384,
 2775,
 531,
 3205,
 113,
 2707,
 460,
 1637,
 3014,
 1398,
 1798,
 3210,
 1974,
 2721,
 3149,
 3205,
 1868,
 839,
 2749,
 1168,
 208,
 3205,
 2793,
 827,
 2721,
 3150,
 799,
 3205,
 393,
 2721,
 1192,
 113,
 2722,
 3206]